# Clustering

In this session, we will learn how to implement some commonly used clustering techniques. <br>
The following topics will be covered:
<ul>
    <li> k-means clustering </li>
    <li> Hierarchical Clustering </li>
    <li> Spectral Clustering </li>
    <li> Metrics to measure performance </li>

</ul>
   

In [ ]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='white', context='notebook', rc={'figure.figsize':(8,8)})

# K-means clustering

The goal of the k-means algorithm is to separate the given $N$ samples $X$ into $K$ clusters, $C$. It achieves this by measuring the dissimilarity between clusters, also known as intertia:

<center>$\Large\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$</center>

In [ ]:
# Making data
from sklearn.datasets import make_blobs, make_circles, make_moons

seed = 42

blob_x, blob_y = make_blobs(n_samples=1000, centers=4, random_state=seed)
circle_x, circle_y = make_circles(n_samples=1000, noise=0.05, factor=0.5, random_state=seed)
moon_x, moon_y = make_moons(n_samples=1000, noise=0.05, random_state=seed)

fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0,0].set_title("Blobs", size=14)
sns.scatterplot(x=blob_x[:,0], y=blob_x[:,1], hue=blob_y, ax=ax[0,0])


ax[0,1].set_title("Circles", size=14)
sns.scatterplot(x=circle_x[:,0], y=circle_x[:,1],  hue=circle_y, ax=ax[0,1])


ax[1,0].set_title("Moons", size=14)
sns.scatterplot(x=moon_x[:,0], y=moon_x[:,1], hue=moon_y, ax=ax[1,0])

ax[1,1].remove()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=2,
    random_state = seed
)

blob_pred = kmeans.fit_predict(blob_x)
circle_pred = kmeans.fit_predict(circle_x)
moon_pred = kmeans.fit_predict(moon_x)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax[0,0].set_title("Blobs", size=14)
sns.scatterplot(x=blob_x[:,0], y=blob_x[:,1], hue=blob_pred, ax=ax[0,0])

ax[0,1].set_title("Circles", size=14)
sns.scatterplot(x=circle_x[:,0], y=circle_x[:,1],  hue=circle_pred, ax=ax[0,1])

ax[1,0].set_title("Moons", size=14)
sns.scatterplot(x=moon_x[:,0], y=moon_x[:,1], hue=moon_pred, ax=ax[1,0])


ax[1,1].remove()

# Scoring a clustering

There are two major ways we can score a clustering:

<ol>
    <li> If we have true values, we check for the concurrence between the predicted clusters and the true clusters. </li>
    <li> In the absence of of true values, we must check for the <i>consistency</i> of the clusters </li>
</ol>
<br>
<u><b> Rand score: </b></u> <br>
Rand index, essentially, computes the similarity between the predicted clusters and the true clusters. It ranges between 0 and 1. It is computed as: <br> <br>

<center> $ \text{RI} = \Large\frac{a + b}{C_2^{n_{samples}}}$ </center>

<ul>
    <li><b>a</b>: the number of pairs of elements that are in the same set in the true cluster and in the same set in predicted cluster.</li>
    <li><b>b</b>: the number of pairs of elements that are in different sets in true cluster and in different sets in predicted cluster. </li>
</ul>

<u><b> Silhouette score:</b></u><br>
Silhouette score explains how well the clusters are separated from one another. It is defined as: <br><br>

<center> $\text{Silhouette score} = \Large \frac{b - a}{max(a, b)}$ </center>

<ul>
    <li><b>a</b>: The mean distance between a sample and all other points in the same class.</li>
    <li><b>b</b>: The mean distance between a sample and all other points in the <i>next nearest cluster</i>. </li>
</ul>
The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.

In [ ]:
# Scoring a clustering

from sklearn.metrics import rand_score, silhouette_score, silhouette_samples

print("Rand_score")
print("Blobs: {:.4f}".format(rand_score(blob_y, blob_pred)))
print("Circles: {:.4f}".format(rand_score(circle_y, circle_pred)))
print("Moons: {:.4f}".format(rand_score(moon_y, moon_pred)))
print("\n")

print("Silhouette_score")
print("Blobs: {:.4f}".format(silhouette_score(blob_x, blob_pred)))
print("Circles: {:.4f}".format(silhouette_score(circle_x, circle_pred)))
print("Moons: {:.4f}".format(silhouette_score(moon_x, moon_pred)))

## Choosing the optimal k value

In [ ]:
# Perform silhouette analysis
# With Blob data
silhouette_scores = []

for k in range(2, 10):
    sample_clusters = KMeans(n_clusters=k, random_state=42).fit_predict(blob_x)
    silhouette_scores.append(
        np.mean(silhouette_samples(blob_x, sample_clusters))
    )

plt.bar(np.arange(2,10), silhouette_scores)
plt.xlabel("# Clusters - K")
plt.ylabel("Sillhouette Score")
plt.title("Silhouette Analysis - Blobs")

# Hierarchical Clustering

The basic idea behind hierarchical clustering is to build nested clusters of samples. For example, take the following cities: <b>Chennai, Trichy, Madurai, Bengaluru, Mysuru</b>. <br>

Two different methods are possible:
<ul>
    <li> Agglomerative Clustering </li>
    <li> Divisive Clustering </li>
</ul>

The criteria by which we decide on whether or not we merge clusters: <b>Linkage</b>. Linkage can be understood as a distance metric between two <i>clusters</i>. In the Scikit-learn implementation, 4 types of linkages are available:

<ul>
    <li><b>Maximum</b> or complete linkage minimizes the maximum distance between observations of pairs of clusters.</li>
    <li><b>Average</b> linkage minimizes the average of the distances between all observations of pairs of clusters.</li>
    <li><b>Single</b> linkage minimizes the distance between the closest observations of pairs of clusters.</li>    
    <li><b>Ward</b> linkage minimizes the sum of squared differences within all clusters. </li>
</ul>

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
clust = AgglomerativeClustering(
    n_clusters=None,               # Needs to be None if distance_threshold is used
    metric = 'euclidean',
    linkage = 'single',
    distance_threshold = 0.125     # Select based on distance between points
)

blob_pred = clust.fit_predict(blob_x, blob_y)
circle_pred = clust.fit_predict(circle_x, circle_y)
moon_pred = clust.fit_predict(moon_x, moon_y)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax[0,0].set_title("Blobs", size=14)
sns.scatterplot(x=blob_x[:,0], y=blob_x[:,1], hue=blob_pred, ax=ax[0,0])

ax[0,1].set_title("Circles", size=14)
sns.scatterplot(x=circle_x[:,0], y=circle_x[:,1],  hue=circle_pred, ax=ax[0,1])

ax[1,0].set_title("Moons", size=14)
sns.scatterplot(x=moon_x[:,0], y=moon_x[:,1], hue=moon_pred, ax=ax[1,0])


ax[1,1].remove()

In [ ]:
print("Rand_score")
print("Blobs: {:.4f}".format(rand_score(blob_y, blob_pred)))
print("Circles: {:.4f}".format(rand_score(circle_y, circle_pred)))
print("Moons: {:.4f}".format(rand_score(moon_y, moon_pred)))
print("\n")

print("Silhouette_score")
print("Blobs: {:.4f}".format(silhouette_score(blob_x, blob_pred)))
print("Circles: {:.4f}".format(silhouette_score(circle_x, circle_pred)))
print("Moons: {:.4f}".format(silhouette_score(moon_x, moon_pred)))

In [ ]:
Z = linkage(
    y = blob_x,       # Input
    method='average', # Linkage
    metric='euclidean'
)

print(Z)

fig, ax = plt.subplots()
R = dendrogram(Z, leaf_font_size=2, ax=ax);
ax.set_title("Dendrgoram of the Clustering - Blobs", size=14);

In [ ]:
Z = linkage(
    y = moon_x,
    method='single',
    metric='euclidean'
)

fig, ax = plt.subplots()
R = dendrogram(Z,  ax=ax);
ax.set_title("Dendrgoram of the Clustering - Moons", size=14)

# DBSCAN

DBSCAN - Density Based Scanning

Idea - Clustered regions are those with high sample density. Very Generic Definition.

- Therefore, can identify arbitrarily shaped clusters
- Can also identify outliers, i.e. points which do not fall under any cluster
- Requires two parameters:
    - `min_samples`: Minimum number of points for group to be cluster
    - `eps`: Maximum distance between two points to be considered neighbours

In [ ]:
from sklearn.cluster import DBSCAN

clust = DBSCAN(
    min_samples=5,
    eps=0.15
)

blob_pred = clust.fit_predict(blob_x, blob_y)
circle_pred = clust.fit_predict(circle_x, circle_y)
moon_pred = clust.fit_predict(moon_x, moon_y)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax[0,0].set_title("Blobs", size=14)
sns.scatterplot(x=blob_x[:,0], y=blob_x[:,1], hue=blob_pred, ax=ax[0,0])

ax[0,1].set_title("Circles", size=14)
sns.scatterplot(x=circle_x[:,0], y=circle_x[:,1],  hue=circle_pred, ax=ax[0,1])

ax[1,0].set_title("Moons", size=14)
sns.scatterplot(x=moon_x[:,0], y=moon_x[:,1], hue=moon_pred, ax=ax[1,0])


ax[1,1].remove()

In [ ]:
print("Rand_score")
print("Blobs: {:.4f}".format(rand_score(blob_y, blob_pred)))
print("Circles: {:.4f}".format(rand_score(circle_y, circle_pred)))
print("Moons: {:.4f}".format(rand_score(moon_y, moon_pred)))
print("\n")

print("Silhouette_score")
print("Blobs: {:.4f}".format(silhouette_score(blob_x, blob_pred)))
print("Circles: {:.4f}".format(silhouette_score(circle_x, circle_pred)))
print("Moons: {:.4f}".format(silhouette_score(moon_x, moon_pred)))